# 연봉예측 머신러닝 모델 생성

**개요**  
학과수업 “머신러닝” 기말과제   
연봉예측 머신러닝 모델 개발 및  캐글 private leaderboard 컴페티션 진행 


**데이터**  
기업제공 인사정보 데이터

**역할**  
조장역할  
컬럼 전처리(직무태그, 세부직종, 어학시험, 자격증)  
GridSeach 이용 파라미터 세부조정  

**진행세부내용**   
1.각 컬럼 전처리  
2.파이프라인을 통한 세부 처리 과정(정규화, 범주형/연속형 분리, 인코딩)간소화  
3.예측모델 Catboost 사용, 평가척도 RMSE  
4.Papermill 코드를 통한 파라미터 세부조작  
5.power mean ensemble을 통해 각 submission 앙상블   

# import

In [296]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
pd.set_option("display.max_columns", 30)

import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV, ShapleyImportanceEvaluator
from optuna.integration.lightgbm import LightGBMTunerCV

import sklearn

from sklearn import set_config
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import cross_validate, GridSearchCV, KFold
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import TargetEncoder, BinaryEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.feature_selection import SelectPercentile, SelectKBest
from sklearn.preprocessing import StandardScaler, PowerTransformer, FunctionTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error, roc_auc_score

from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.svm import SVC
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import LinearRegression, LogisticRegression
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


import warnings
warnings.filterwarnings("ignore")

import klib
from matplotlib import rc

#ignore warningss
import warnings
warnings.filterwarnings('ignore')

rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

In [297]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [298]:
X_train = pd.read_csv("/content/drive/MyDrive/머신러닝 수업/data/X_train.csv", encoding="CP949")
X_test = pd.read_csv("/content/drive/MyDrive/머신러닝 수업/data/X_test.csv", encoding="CP949")
y_train = pd.read_csv("/content/drive/MyDrive/머신러닝 수업/data/y_train.csv", encoding="CP949")

In [299]:
X_train.columns

Index(['ID', '직종', '세부직종', '직무태그', '근무경력', '근무형태', '근무지역', '출신대학', '대학전공',
       '어학시험', '자격증', '대학성적'],
      dtype='object')

In [300]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16570 entries, 0 to 16569
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      16570 non-null  object 
 1   직종      16570 non-null  object 
 2   세부직종    16570 non-null  object 
 3   직무태그    14055 non-null  object 
 4   근무경력    16570 non-null  object 
 5   근무형태    6661 non-null   object 
 6   근무지역    16570 non-null  object 
 7   출신대학    16570 non-null  object 
 8   대학전공    16570 non-null  object 
 9   어학시험    4988 non-null   object 
 10  자격증     16570 non-null  object 
 11  대학성적    14600 non-null  float64
dtypes: float64(1), object(11)
memory usage: 1.5+ MB


# 전처리

## 직무태그

In [301]:
X_train.직무태그 = X_train.직무태그.fillna("기타")

In [302]:
X_train.직무태그 = X_train.직무태그.apply(lambda x : ''.join(filter(str.isalnum, x))) #특문없애기
X_train.직무태그 = X_train.직무태그.apply(lambda x : re.sub(r"[0-9]", "", x)) #숫자없애기
X_train.직무태그 = X_train.직무태그.str.lower() #대문자 > 소문자
X_train.직무태그= X_train.직무태그.replace('','기타') #빈값 기타로 변경
X_train = X_train.sort_values(by='직무태그')

In [303]:
#웹관련 직종 "웹"으로 모두 변경

웹=["웹",
"웹pd",
"웹pm",
"웹ui",
"웹개발",
"웹개발자",
"웹관리",
"웹기획",
"웹기획과헤드헌팅을경력자",
"웹기획및웹마케팅",
"웹기획웹pd웹pm웹si기획",
"웹기획웹마스터",
"웹디자이너",
"웹디자인",
"웹디자인게임사이트프로모션",
"웹디자인플래쉬일러스트레이터포토샵드림위버디자이너",
"웹디지인",
"웹마스타",
"웹마스터",
"웹마케팅",
"웹사이트관리",
"웹사이트기획",
"웹사이트운영",
"웹사이트운영관리",
"웹사이트운영및관리",
"웹서버관리",
"웹서비스",
"웹서비스기획",
"웹솔루션",
"웹어플리케이션개발",
"웹에이전시",
"웹에이젼시",
"웹프로",
"웹프로그래머",
"웹프로그래머asp",
"웹프로그래머web",
"웹프로그래밍",
"웹프로그램",
"웹프로그램java",
"웹프로그램머",
"웹프로그램머한규"]

In [304]:
X_train.직무태그 = X_train.직무태그.replace(웹,'웹')

In [305]:
X_train.직무태그 = X_train.직무태그.str.split('').str[1:3].str.join('') #앞 두 글자만 분류

In [306]:
train직무=pd.DataFrame(X_train.직무태그.value_counts()).reset_index()

In [307]:
train직무리스트=list(train직무[train직무['count'] == 1]['직무태그'])

In [308]:
for i in range(len(X_train)):
    if X_train.loc[i]['직무태그'] in train직무리스트 :
        X_train.loc[i,'직무태그'] = '기타'

In [309]:
# test직무태그

In [310]:
X_test.직무태그 = X_test.직무태그.fillna("기타")
X_test.직무태그 = X_test.직무태그.str.split(',').str[0]
X_test.직무태그 = X_test.직무태그.apply(lambda x : ''.join(filter(str.isalnum, x))) #특문없애기
X_test.직무태그 = X_test.직무태그.apply(lambda x : re.sub(r"[0-9]", "", x)) #숫자없애기
X_test.직무태그 = X_test.직무태그.str.lower() #대문자 > 소문자
X_test.직무태그= X_test.직무태그.replace('','기타') #빈값 기타로 변경
X_test = X_test.sort_values(by='직무태그')
#pd.DataFrame(list(X_test.직무태그.unique())).to_csv("test직무태그.csv", encoding="CP949") #엑셀로 종류 자세히 확인

In [311]:
웹2=["웹",
"웹md",
"웹pd",
"웹ui",
"웹개발",
"웹기획",
"웹기획마케팅",
"웹기획및개발기획",
"웹기획웹프로젝트어린이사이트교육컨텐츠대입사이트",
"웹기획이벤트및서비스",
"웹디자이너",
"웹디자인",
"웹디자인디자이너플래셔플래시",
"웹마스터",
"웹마스터linux",
"웹마케팅",
"웹사이트운영",
"웹서버관리asp",
"웹서비스기획",
"웹서퍼",
"웹컨텐츠기획및개발",
"웹퍼블리셔",
"웹프로그래머",
"웹프로그래머jsp서블릿일본어",
"웹프로그래밍",
"웹프로그래밍asp",
"웹프로그램",
"웹프로그램개발",
"웹프로그램어",
"웹프로래머",
"웹프로모션"]

In [312]:
X_test.직무태그 = X_test.직무태그.replace(웹,'웹')
X_test.직무태그 = X_test.직무태그.str.split('').str[1:3].str.join('') #앞 두 글자만 분류

In [313]:
test직무=pd.DataFrame(X_test.직무태그.value_counts()).reset_index()

In [314]:
test직무리스트=list(test직무[test직무['count'] == 1]['직무태그'])

In [315]:
for i in range(len(X_test)):
    if X_test.loc[i]['직무태그'] in test직무리스트 :
        X_test.loc[i,'직무태그'] = '기타'

## 어학시험

In [316]:
# 어학시험 train

In [317]:
X_train.어학시험.value_counts()

어학시험
TOEIC             2767
                  1298
JLPT               289
HSK                197
JPT                108
TOEFL               80
기타                  48
OPIc                43
TEPS                27
TOEIC Speaking      22
IELTS               22
TOEFL-CBT           21
G-TELP              19
JTRA                 9
기타시험                 9
TOEFL-IBT            8
新HSK                 7
SEPT                 3
TOEFL-PBT            3
G-ETAT               2
G-TELP(GLT)          1
DELF                 1
OPI                  1
GRE                  1
DALF                 1
TOEIC S&W            1
Name: count, dtype: int64

In [318]:
X_train.어학시험 = X_train.어학시험.replace(' ','없음')

In [319]:
X_train.어학시험 = X_train.어학시험.fillna('없음')

In [320]:
c=pd.DataFrame(X_train.어학시험.value_counts()).reset_index()

In [321]:
어학시험10미만=list(c[c['count'] < 10].어학시험)

In [322]:
X_train.어학시험 = X_train.어학시험.replace(어학시험10미만,'기타')

In [323]:
# 어학시험 test

In [324]:
X_test.어학시험 = X_test.어학시험.replace(' ','없음')
X_test.어학시험 = X_test.어학시험.fillna('없음')
c=pd.DataFrame(X_test.어학시험.value_counts()).reset_index()
어학시험10미만=list(c[c['count'] < 10].어학시험)
X_test.어학시험 = X_test.어학시험.replace(어학시험10미만,'기타')

## 근무형태

In [325]:
X_train['근무형태'].value_counts(normalize=True, dropna=False)

근무형태
NaN                                0.598008
정규직                                0.339650
정규직, 계약직                           0.019734
정규직, 해외취업                          0.011648
정규직, 계약직,                          0.003742
계약직                                0.002957
정규직, 계약직, 인턴                       0.002957
정규직,                               0.002836
정규직, 계약직, 해외취업                     0.002655
정규직, 인턴                            0.002354
정규직, 계약직, 파견직                      0.001811
정규직, 계약직, 해외취업,                    0.001509
정규직, 계약직, 해외취업, 파견직                0.000966
정규직, 계약직, 해외취업, 파견직,               0.000785
계약직,                               0.000785
정규직, 계약직, 파견직,                     0.000724
정규직, 파견직                           0.000724
정규직, 계약직, 해외취업, 인턴                 0.000604
정규직, 해외취업, 파견직                     0.000543
정규직, 계약직, 인턴, 파견직                  0.000422
정규직, 해외취업,                         0.000422
해외취업                               0.000362
정규직, 병역특례                  

In [326]:
# 가장 앞의 기록만 남김
X_train['근무형태'] = X_train['근무형태'].str.split(', ').str[0]

> test data도 동일하게 처리

In [327]:
X_test['근무형태'] = X_test['근무형태'].str.split(', ').str[0]

In [328]:
X_test['근무형태'].value_counts(normalize=True, dropna=False)

근무형태
NaN     0.606173
정규직     0.390297
계약직     0.002625
해외취업    0.000362
파견직     0.000272
인턴      0.000181
병역특례    0.000091
Name: proportion, dtype: float64

## 근무지역

In [329]:
X_train['근무지역'].value_counts(normalize=True, dropna=False)

근무지역
서울,경기,      0.136391
서울,,        0.121243
서울,서울,서울    0.112251
서울,경기,인천    0.041038
서울,서울,경기    0.031684
              ...   
강원,경기,기타    0.000060
울산,서울,경남    0.000060
해외,부산,서울    0.000060
전북,경기,충남    0.000060
부산,부산,경기    0.000060
Name: proportion, Length: 1313, dtype: float64

In [330]:
# 가장 앞의 기록만 남김
X_train['근무지역'] = X_train['근무지역'].str.split(',').str[0]

In [331]:
# 해외 지역은 모두 '해외' 하나로 묶음
abroad = ['중국','일본','미국','인도네시아','홍콩','싱가포르','대만','말레이시아','캐나다','필리핀']
X_train['근무지역'] = X_train.replace(abroad, '해외')['근무지역']

In [332]:
# 전국은 '기타'로
X_train['근무지역'] = X_train.replace('전국', '기타')['근무지역']

> test data도 동일하게 처리

In [333]:
X_test['근무지역'] = X_test['근무지역'].str.split(',').str[0]
X_test['근무지역'] = X_test.replace(abroad, '해외')['근무지역']
X_test['근무지역'] = X_test.replace('전국', '기타')['근무지역']

In [334]:
X_test['근무지역'].value_counts(normalize=True, dropna=False)

근무지역
서울     0.766293
경기     0.065080
부산     0.051231
인천     0.018012
대전     0.017198
해외     0.015025
기타     0.013125
광주     0.011586
전북     0.009051
충남     0.008689
경남     0.005612
충북     0.005612
울산     0.002806
제주     0.002444
강원     0.002444
대구     0.002172
경북     0.002082
전남     0.001177
러시아    0.000181
프랑스    0.000091
인도     0.000091
Name: proportion, dtype: float64

In [335]:
# [러시아, 프랑스, 인도]가 train과 다름
abroad = ['중국','일본','미국','인도네시아','홍콩','싱가포르','대만','말레이시아','캐나다','필리핀','러시아','프랑스','인도']
X_test['근무지역'] = X_test.replace(abroad, '해외')['근무지역']

## 근무경력

In [336]:
X_train['근무경력'].value_counts(dropna=False)

근무경력
0개월        9145
2년 11개월     281
3년 0개월      209
5년 1개월      190
1년 0개월      177
           ... 
19년 4개월       1
21년 5개월       1
19년 1개월       1
26년 6개월       1
27년 0개월       1
Name: count, Length: 311, dtype: int64

---
- 연 단위에서 끊겨도 n년 0개월로 표시됨
- 따라서 'n년 l개월', 혹은 'm개월' 밖에 없음

In [337]:
# 개월 단위 (l개월, m개월)
X_train['근무경력'].str.split('년 ').str[-1]

1679     0개월
5182     0개월
3116     6개월
8390     7개월
4496     0개월
        ... 
1097     4개월
15843    2개월
5385     0개월
11019    1개월
3137     0개월
Name: 근무경력, Length: 16570, dtype: object

In [338]:
# 연 단위
X_train['근무경력'].str.split('년 ').str[0]
# -> n(of n년 l개월)과 m개월 출력됨..!

1679     0개월
5182     0개월
3116       3
8390       4
4496     0개월
        ... 
1097       1
15843    2개월
5385     0개월
11019     11
3137     0개월
Name: 근무경력, Length: 16570, dtype: object

In [339]:
# 연 단위, 개월 단위 열 별도로 기록
X_train['year'] = X_train['근무경력'].str.split('년 ').str[-1] != X_train['근무경력'].str.split('년 ').str[0]
X_train['month'] = X_train['근무경력'].str.split('년 ').str[-1] == X_train['근무경력'].str.split('년 ').str[0]

In [340]:
X_train['month']

1679      True
5182      True
3116     False
8390     False
4496      True
         ...  
1097     False
15843     True
5385      True
11019    False
3137      True
Name: month, Length: 16570, dtype: bool

> m개월 처리

In [341]:
# 개월 단위 index에 해당하는 행 반환, 숫자만 분리
X_train.loc[X_train['month']==True]['근무경력'].str.split('개월').str[0]

1679     0
5182     0
4496     0
7954     0
9308     0
        ..
3944     0
7375     0
15843    2
5385     0
3137     0
Name: 근무경력, Length: 9406, dtype: object

In [342]:
# update
X_train.loc[X_train['month']==True, '근무경력'] = X_train.loc[X_train['month']==True]['근무경력'].str.split('개월').str[0].astype(int)*30

> n년 l개월 처리
- n년 l개월은 (n x 365) + (l x 30) 해줄 것임

In [343]:
# n년->일
n = X_train.loc[X_train['year']==True]['근무경력'].str.split('년 ').str[0].astype(int)*365

In [344]:
# l개월->일
l = X_train.loc[X_train['year']==True]['근무경력'].str.split('년 ').str[-1].str.split('개월').str[0].astype(int)*30

In [345]:
n+l

3116     1275
8390     1670
15464    4745
14711    1095
12681    1700
         ... 
16464    1215
13861    3010
9146      515
1097      485
11019    4045
Name: 근무경력, Length: 7164, dtype: int64

In [346]:
# update
X_train.loc[X_train['year']==True, '근무경력'] = n+l

In [347]:
drop_list = ['year','month']
X_train = X_train.drop(drop_list, axis=1)

In [348]:
X_train['근무경력'] = X_train['근무경력'].astype(int)
X_train[['근무경력']].describe()

,근무경력
count,16570.000000
mean,1060.632468
std,1848.384332
min,0.000000
25%,0.000000
50%,0.000000
75%,1700.000000
max,43950.000000


In [349]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16570 entries, 1679 to 3137
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      16570 non-null  object 
 1   직종      16570 non-null  object 
 2   세부직종    16570 non-null  object 
 3   직무태그    16570 non-null  object 
 4   근무경력    16570 non-null  int64  
 5   근무형태    6661 non-null   object 
 6   근무지역    16570 non-null  object 
 7   출신대학    16570 non-null  object 
 8   대학전공    16570 non-null  object 
 9   어학시험    16570 non-null  object 
 10  자격증     16570 non-null  object 
 11  대학성적    14600 non-null  float64
dtypes: float64(1), int64(1), object(10)
memory usage: 2.1+ MB


> test data도 동일하게 처리

In [350]:
X_test['year'] = X_test['근무경력'].str.split('년 ').str[-1] != X_test['근무경력'].str.split('년 ').str[0]
X_test['month'] = X_test['근무경력'].str.split('년 ').str[-1] == X_test['근무경력'].str.split('년 ').str[0]

# month * 30
X_test.loc[X_test['month']==True, '근무경력'] = X_test.loc[X_test['month']==True]['근무경력'].str.split('개월').str[0].astype(int)*30
# (year*365) + (month*30)
n = X_test.loc[X_test['year']==True]['근무경력'].str.split('년 ').str[0].astype(int)*365
l = X_test.loc[X_test['year']==True]['근무경력'].str.split('년 ').str[-1].str.split('개월').str[0].astype(int)*30
X_test.loc[X_test['year']==True, '근무경력'] = n+l

X_test = X_test.drop(drop_list, axis=1)
X_test['근무경력'] = X_test['근무경력'].astype(int)

X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11048 entries, 1114 to 1769
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      11048 non-null  object 
 1   직종      11048 non-null  object 
 2   세부직종    11048 non-null  object 
 3   직무태그    11048 non-null  object 
 4   근무경력    11048 non-null  int64  
 5   근무형태    4351 non-null   object 
 6   근무지역    11048 non-null  object 
 7   출신대학    11048 non-null  object 
 8   대학전공    11048 non-null  object 
 9   어학시험    11048 non-null  object 
 10  자격증     11048 non-null  object 
 11  대학성적    9730 non-null   float64
dtypes: float64(1), int64(1), object(10)
memory usage: 1.3+ MB


In [351]:
X_test[['근무경력']].describe()

,근무경력
count,11048.000000
mean,1078.191528
std,1762.551012
min,0.000000
25%,0.000000
50%,0.000000
75%,1700.000000
max,28405.000000


## 세부직종

In [352]:
job_count_df = X_train['세부직종'].value_counts().to_frame().reset_index()

In [353]:
job_under_10 = list(job_count_df.loc[job_count_df['count']<10].세부직종)

In [354]:
X_train = X_train.replace({'세부직종':job_under_10}, '기타')

> test data도 동일하게 처리

In [355]:
job_count_df_tst = X_test['세부직종'].value_counts().to_frame().reset_index()

In [356]:
job_under_10_tst = list(job_count_df.loc[job_count_df['count']<10].세부직종)

In [357]:
X_test = X_test.replace({'세부직종':job_under_10_tst}, '기타')

## 대학전공

In [358]:
X_train_pre = X_test.copy()

In [359]:
X_train['대학전공'].value_counts()

대학전공
경영학           774
경영학과          450
컴퓨터공학         322
경제학           322
전자공학          243
             ... 
인터넷소프트웨어학과      1
응용생물과학          1
전기공학부           1
식품생명자원학과        1
SOC             1
Name: count, Length: 2030, dtype: int64

In [360]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("경영정보")]
a.loc[:, '대학전공'] = '경영정보'
X_train_pre[X_train_pre['대학전공'].str.contains("경영정보")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("경영정보")]['대학전공'].value_counts()

대학전공
경영정보    78
Name: count, dtype: int64

In [361]:
ac = X_train_pre[X_train_pre['대학전공'].str.contains("회계")]
ac.loc[:, '대학전공'] = '회계'
X_train_pre[X_train_pre['대학전공'].str.contains("회계")] = ac
X_train_pre[X_train_pre['대학전공'].str.contains("회계")]['대학전공'].value_counts()

대학전공
회계    326
Name: count, dtype: int64

In [362]:
ap = X_train_pre[X_train_pre['대학전공'].str.contains("홍보")]
ap.loc[:, '대학전공'] = '광고홍보'
X_train_pre[X_train_pre['대학전공'].str.contains("홍보")] = ap
X_train_pre[X_train_pre['대학전공'].str.contains("홍보")]['대학전공'].value_counts()

대학전공
광고홍보    64
Name: count, dtype: int64

In [363]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("게임")]
a.loc[:, '대학전공'] = '게임'
X_train_pre[X_train_pre['대학전공'].str.contains("게임")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("게임")]['대학전공'].value_counts()

대학전공
게임    6
Name: count, dtype: int64

In [364]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("소프트웨어")]
a.loc[:, '대학전공'] = '소프트웨어'
X_train_pre[X_train_pre['대학전공'].str.contains("소프트웨어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("소프트웨어")]['대학전공'].value_counts()

대학전공
소프트웨어    6
Name: count, dtype: int64

In [365]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("심리")]
a.loc[:, '대학전공'] = '심리'
X_train_pre[X_train_pre['대학전공'].str.contains("심리")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("심리")]['대학전공'].value_counts()

대학전공
심리    61
Name: count, dtype: int64

In [366]:
a = X_train_pre[X_train_pre['대학전공'].str.startswith("교육")]
a.loc[:, '대학전공'] = '교1육'
X_train_pre[X_train_pre['대학전공'].str.startswith("교육")] = a
X_train_pre[X_train_pre['대학전공'].str.startswith("교1육")]['대학전공'].value_counts()

대학전공
교1육    42
Name: count, dtype: int64

In [367]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("교육")]
a.loc[:, '대학전공'] = '사범'
X_train_pre[X_train_pre['대학전공'].str.contains("교육")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("사범")]['대학전공'].value_counts()

대학전공
사범     123
사범대      1
Name: count, dtype: int64

In [368]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("사범")]
a.loc[:, '대학전공'] = '사범'
X_train_pre[X_train_pre['대학전공'].str.contains("사범")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("사범")]['대학전공'].value_counts()

대학전공
사범    124
Name: count, dtype: int64

In [369]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("공예")]
a.loc[:, '대학전공'] = '공예'
X_train_pre[X_train_pre['대학전공'].str.contains("공예")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("공예")]['대학전공'].value_counts()

대학전공
공예    51
Name: count, dtype: int64

In [370]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("의류")]
a.loc[:, '대학전공'] = '의류'
X_train_pre[X_train_pre['대학전공'].str.contains("의류")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("의류")]['대학전공'].value_counts()

대학전공
의류    207
Name: count, dtype: int64

In [371]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("패션")]
a.loc[:, '대학전공'] = '패션'
X_train_pre[X_train_pre['대학전공'].str.contains("패션")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("패션")]['대학전공'].value_counts()

대학전공
패션    137
Name: count, dtype: int64

In [372]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("소비자")]
a.loc[:, '대학전공'] = '소비자'
X_train_pre[X_train_pre['대학전공'].str.contains("소비자")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("소비자")]['대학전공'].value_counts()

대학전공
소비자    15
Name: count, dtype: int64

In [373]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("경찰")]
a.loc[:, '대학전공'] = '경찰'
X_train_pre[X_train_pre['대학전공'].str.contains("경찰")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("경찰")]['대학전공'].value_counts()

대학전공
경찰    7
Name: count, dtype: int64

In [374]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("행정")]
a.loc[:, '대학전공'] = '행정'
X_train_pre[X_train_pre['대학전공'].str.contains("행정")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("행정")]['대학전공'].value_counts()

대학전공
행정    237
Name: count, dtype: int64

In [375]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("복지")]
a.loc[:, '대학전공'] = '복지'
X_train_pre[X_train_pre['대학전공'].str.contains("복지")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("복지")]['대학전공'].value_counts()

대학전공
복지    69
Name: count, dtype: int64

In [376]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("통계")]
a.loc[:, '대학전공'] = '통계'
X_train_pre[X_train_pre['대학전공'].str.contains("통계")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("통계")]['대학전공'].value_counts()

대학전공
통계    217
Name: count, dtype: int64

In [377]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("스포츠")]
a.loc[:, '대학전공'] = '스포츠'
X_train_pre[X_train_pre['대학전공'].str.contains("스포츠")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("스포츠")]['대학전공'].value_counts()

대학전공
스포츠    11
Name: count, dtype: int64

In [378]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("전자계")]
a.loc[:, '대학전공'] = '컴퓨터'
X_train_pre[X_train_pre['대학전공'].str.contains("전자계")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("컴퓨터")]['대학전공'].value_counts()

대학전공
컴퓨터            207
컴퓨터공학          200
컴퓨터공학과         100
컴퓨터학과           34
컴퓨터과학           25
              ... 
컴퓨터수리정보          1
컴퓨터공학전공          1
컴퓨터학과, 물리학과      1
컴퓨터정보통신공학        1
컴퓨터응용과학과         1
Name: count, Length: 70, dtype: int64

In [379]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("재료")]
a.loc[:, '대학전공'] = '재료공'
X_train_pre[X_train_pre['대학전공'].str.contains("재료")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("재료공")]['대학전공'].value_counts()

대학전공
재료공    80
Name: count, dtype: int64

In [380]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("방송")]
a.loc[:, '대학전공'] = '방송'
X_train_pre[X_train_pre['대학전공'].str.contains("방송")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("방송")]['대학전공'].value_counts()

대학전공
방송    123
Name: count, dtype: int64

In [381]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("통신")]
a.loc[:, '대학전공'] = '통신'
X_train_pre[X_train_pre['대학전공'].str.contains("통신")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("통신")]['대학전공'].value_counts()

대학전공
통신    236
Name: count, dtype: int64

In [382]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("전자")]
a.loc[:, '대학전공'] = '전자'
X_train_pre[X_train_pre['대학전공'].str.contains("전자")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("전자")]['대학전공'].value_counts()

대학전공
전자    393
Name: count, dtype: int64

In [383]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("건축")]
a.loc[:, '대학전공'] = '건축'
X_train_pre[X_train_pre['대학전공'].str.contains("건축")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("건축")]['대학전공'].value_counts()

대학전공
건축    136
Name: count, dtype: int64

In [384]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("의상")]
a.loc[:, '대학전공'] = '의상디1자인'
X_train_pre[X_train_pre['대학전공'].str.contains("의상")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("의상디1자인")]['대학전공'].value_counts()

대학전공
의상디1자인    136
Name: count, dtype: int64

In [385]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("시각")]
a.loc[:, '대학전공'] = '시각디1자인'
X_train_pre[X_train_pre['대학전공'].str.contains("시각")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("시각디1자인")]['대학전공'].value_counts()

대학전공
시각디1자인    92
Name: count, dtype: int64

In [386]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("산업디")]
a.loc[:, '대학전공'] = '산업디1자인'
X_train_pre[X_train_pre['대학전공'].str.contains("산업디")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("산업디1자인")]['대학전공'].value_counts()

대학전공
산업디1자인    160
Name: count, dtype: int64

In [387]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("경제")]
a.loc[:, '대학전공'] = '경제'
X_train_pre[X_train_pre['대학전공'].str.contains("경제")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("경제")]['대학전공'].value_counts()

대학전공
경제    421
Name: count, dtype: int64

In [388]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("통상")]
a.loc[:, '대학전공'] = '국제통상'
X_train_pre[X_train_pre['대학전공'].str.contains("통상")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("국제통상")]['대학전공'].value_counts()

대학전공
국제통상    50
Name: count, dtype: int64

In [389]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("무역")]
a.loc[:, '대학전공'] = '국제통상'
X_train_pre[X_train_pre['대학전공'].str.contains("무역")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("국제통상")]['대학전공'].value_counts()

대학전공
국제통상    345
Name: count, dtype: int64

In [390]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("일어일")]
a.loc[:, '대학전공'] = '일본'
X_train_pre[X_train_pre['대학전공'].str.contains("일어일")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("일본")]['대학전공'].value_counts()

대학전공
일본              139
일본어              36
일본어학과             6
일본학과              6
일본학               5
일본어학              2
일본어과              2
일본어 문학            1
관광일본어             1
동양어문학부 일본어과       1
일어 일본학과           1
일본어 학과            1
관광일본어학과           1
관광경영,일본어 부전공      1
Name: count, dtype: int64

In [391]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("일본")]
a.loc[:, '대학전공'] = '일본'
X_train_pre[X_train_pre['대학전공'].str.contains("일본")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("일본")]['대학전공'].value_counts()

대학전공
일본    203
Name: count, dtype: int64

In [392]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("일문")]
a.loc[:, '대학전공'] = '일본'
X_train_pre[X_train_pre['대학전공'].str.contains("일문")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("일본")]['대학전공'].value_counts()

대학전공
일본    203
Name: count, dtype: int64

In [393]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("중어")]
a.loc[:, '대학전공'] = '중국'
X_train_pre[X_train_pre['대학전공'].str.contains("중어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("중국")]['대학전공'].value_counts()

대학전공
중국        136
중국어        36
중국학        20
중국학과        9
중국어학        5
중국어학과       4
중국어과        3
관광중국어       1
중국사         1
중국어중국학      1
중국언어문화      1
Name: count, dtype: int64

In [394]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("중국")]
a.loc[:, '대학전공'] = '중국'
X_train_pre[X_train_pre['대학전공'].str.contains("중국")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("중국")]['대학전공'].value_counts()

대학전공
중국    217
Name: count, dtype: int64

In [395]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("중문")]
a.loc[:, '대학전공'] = '중국'
X_train_pre[X_train_pre['대학전공'].str.contains("중문")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("중국")]['대학전공'].value_counts()

대학전공
중국    222
Name: count, dtype: int64

In [396]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("영어")]
a.loc[:, '대학전공'] = '영어'
X_train_pre[X_train_pre['대학전공'].str.contains("영어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("영어")]['대학전공'].value_counts()

대학전공
영어    404
Name: count, dtype: int64

In [397]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("영문")]
a.loc[:, '대학전공'] = '영어'
X_train_pre[X_train_pre['대학전공'].str.contains("영문")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("영어")]['대학전공'].value_counts()

대학전공
영어    447
Name: count, dtype: int64

In [398]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("국어")]
a.loc[:, '대학전공'] = '국어'
X_train_pre[X_train_pre['대학전공'].str.contains("국어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("국어")]['대학전공'].value_counts()

대학전공
국어    129
Name: count, dtype: int64

In [399]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("국문")]
a.loc[:, '대학전공'] = '국어'
X_train_pre[X_train_pre['대학전공'].str.contains("국문")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("국어")]['대학전공'].value_counts()

대학전공
국어    137
Name: count, dtype: int64

In [400]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("노어")]
a.loc[:, '대학전공'] = '러시아'
X_train_pre[X_train_pre['대학전공'].str.contains("노어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("러시아")]['대학전공'].value_counts()

대학전공
러시아      13
러시아학      5
러시아학과     3
러시아어      2
러시아어/     1
러시아과      1
Name: count, dtype: int64

In [401]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("러시아")]
a.loc[:, '대학전공'] = '러시아'
X_train_pre[X_train_pre['대학전공'].str.contains("러시아")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("러시아")]['대학전공'].value_counts()

대학전공
러시아    25
Name: count, dtype: int64

In [402]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("노문")]
a.loc[:, '대학전공'] = '러시아'
X_train_pre[X_train_pre['대학전공'].str.contains("노문")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("러시아")]['대학전공'].value_counts()

대학전공
러시아    25
Name: count, dtype: int64

In [403]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("불어")]
a.loc[:, '대학전공'] = '프랑스'
X_train_pre[X_train_pre['대학전공'].str.contains("불어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("프랑스")]['대학전공'].value_counts()

대학전공
프랑스        104
프랑스어         1
프랑스어문학과      1
프랑스어학과       1
Name: count, dtype: int64

In [404]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("프랑스")]
a.loc[:, '대학전공'] = '프랑스'
X_train_pre[X_train_pre['대학전공'].str.contains("프랑스")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("프랑스")]['대학전공'].value_counts()

대학전공
프랑스    107
Name: count, dtype: int64

In [405]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("불문")]
a.loc[:, '대학전공'] = '프랑스'
X_train_pre[X_train_pre['대학전공'].str.contains("불문")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("프랑스")]['대학전공'].value_counts()

대학전공
프랑스    113
Name: count, dtype: int64

In [406]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("독어")]
a.loc[:, '대학전공'] = '독일'
X_train_pre[X_train_pre['대학전공'].str.contains("독어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("독일")]['대학전공'].value_counts()

대학전공
독일          83
독일어          5
독일학과         1
독일언어문화학      1
독일언어문화학과     1
Name: count, dtype: int64

In [407]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("독일")]
a.loc[:, '대학전공'] = '독일'
X_train_pre[X_train_pre['대학전공'].str.contains("독일")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("독일")]['대학전공'].value_counts()

대학전공
독일    91
Name: count, dtype: int64

In [408]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("독문")]
a.loc[:, '대학전공'] = '독일'
X_train_pre[X_train_pre['대학전공'].str.contains("독문")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("독일")]['대학전공'].value_counts()

대학전공
독일    98
Name: count, dtype: int64

In [409]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("천문")]
a.loc[:, '대학전공'] = '천문'
X_train_pre[X_train_pre['대학전공'].str.contains("천문")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("천문")]['대학전공'].value_counts()

대학전공
천문    7
Name: count, dtype: int64

In [410]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("체육")]
a.loc[:, '대학전공'] = '체육'
X_train_pre[X_train_pre['대학전공'].str.contains("체육")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("체육")]['대학전공'].value_counts()

대학전공
체육    98
Name: count, dtype: int64

In [411]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("사회")]
a.loc[:, '대학전공'] = '사회'
X_train_pre[X_train_pre['대학전공'].str.contains("사회")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("사회")]['대학전공'].value_counts()

대학전공
사회    76
Name: count, dtype: int64

In [412]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("문학")]
a.loc[:, '대학전공'] = '기타문학'
X_train_pre[X_train_pre['대학전공'].str.contains("문학")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("기타문학")]['대학전공'].value_counts()

대학전공
기타문학    6
Name: count, dtype: int64

In [413]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("사학")]
a.loc[:, '대학전공'] = '사학'
X_train_pre[X_train_pre['대학전공'].str.contains("사학")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("사학")]['대학전공'].value_counts()

대학전공
사학    97
Name: count, dtype: int64

In [414]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("역사")]
a.loc[:, '대학전공'] = '사학'
X_train_pre[X_train_pre['대학전공'].str.contains("역사")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("사학")]['대학전공'].value_counts()

대학전공
사학    102
Name: count, dtype: int64

In [415]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("국사")]
a.loc[:, '대학전공'] = '사학'
X_train_pre[X_train_pre['대학전공'].str.contains("국사")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("사학")]['대학전공'].value_counts()

대학전공
사학    103
Name: count, dtype: int64

In [416]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("사하과")]
a.loc[:, '대학전공'] = '사학'
X_train_pre[X_train_pre['대학전공'].str.contains("사하과")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("사학")]['대학전공'].value_counts()

대학전공
사학    103
Name: count, dtype: int64

In [417]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("기계")]
a.loc[:, '대학전공'] = '기계'
X_train_pre[X_train_pre['대학전공'].str.contains("기계")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("기계")]['대학전공'].value_counts()

대학전공
기계    452
Name: count, dtype: int64

In [418]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("화학공")]
a.loc[:, '대학전공'] = '화1학공1학'
X_train_pre[X_train_pre['대학전공'].str.contains("화학공")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("화1학공1학")]['대학전공'].value_counts()

대학전공
화1학공1학    156
Name: count, dtype: int64

In [419]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("체육")]
a.loc[:, '대학전공'] = '체육'
X_train_pre[X_train_pre['대학전공'].str.contains("체육")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("체육")]['대학전공'].value_counts()

대학전공
체육    98
Name: count, dtype: int64

In [420]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("전기")]
a.loc[:, '대학전공'] = '전기'
X_train_pre[X_train_pre['대학전공'].str.contains("전기")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("전기")]['대학전공'].value_counts()

대학전공
전기    156
Name: count, dtype: int64

In [421]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("디자인")]
a.loc[:, '대학전공'] = '기타 디자인'
X_train_pre[X_train_pre['대학전공'].str.contains("디자인")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("기타 디자인")]['대학전공'].value_counts()

대학전공
기타 디자인    101
Name: count, dtype: int64

In [422]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("만화")]
a.loc[:, '대학전공'] = '만화'
X_train_pre[X_train_pre['대학전공'].str.contains("만화")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("만화")]['대학전공'].value_counts()

Series([], Name: count, dtype: int64)

In [423]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("식품영양")]
a.loc[:, '대학전공'] = '식1품영양'
X_train_pre[X_train_pre['대학전공'].str.contains("식품영양")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("식1품영양")]['대학전공'].value_counts()

대학전공
식1품영양    67
Name: count, dtype: int64

In [424]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("식품")]
a.loc[:, '대학전공'] = '식품공1학'
X_train_pre[X_train_pre['대학전공'].str.contains("식품")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("식품공1학")]['대학전공'].value_counts()

대학전공
식품공1학    69
Name: count, dtype: int64

In [425]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("법")]
a.loc[:, '대학전공'] = '법학'
X_train_pre[X_train_pre['대학전공'].str.contains("법")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("법학")]['대학전공'].value_counts()

대학전공
법학    211
Name: count, dtype: int64

In [426]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("시스템")]
a.loc[:, '대학전공'] = '산업정보시스템'
X_train_pre[X_train_pre['대학전공'].str.contains("시스템")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("산업정보시스템")]['대학전공'].value_counts()

대학전공
산업정보시스템    40
Name: count, dtype: int64

In [427]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("음")]
a.loc[:, '대학전공'] = '음악'
X_train_pre[X_train_pre['대학전공'].str.contains("음")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("음악")]['대학전공'].value_counts()

대학전공
음악    10
Name: count, dtype: int64

In [428]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("철학")]
a.loc[:, '대학전공'] = '철학'
X_train_pre[X_train_pre['대학전공'].str.contains("철학")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("철학")]['대학전공'].value_counts()

대학전공
철학    62
Name: count, dtype: int64

In [429]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("사진")]
a.loc[:, '대학전공'] = '사진'
X_train_pre[X_train_pre['대학전공'].str.contains("사진")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("사진")]['대학전공'].value_counts()

대학전공
사진    5
Name: count, dtype: int64

In [430]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("문헌")]
a.loc[:, '대학전공'] = '문헌정1보'
X_train_pre[X_train_pre['대학전공'].str.contains("문헌")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("문헌정1보")]['대학전공'].value_counts()

대학전공
문헌정1보    79
Name: count, dtype: int64

In [431]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("광고")]
a.loc[:, '대학전공'] = '광고'
X_train_pre[X_train_pre['대학전공'].str.contains("광고")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("광고")]['대학전공'].value_counts()

대학전공
광고    71
Name: count, dtype: int64

In [432]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("언론")]
a.loc[:, '대학전공'] = '커뮤니케이션'
X_train_pre[X_train_pre['대학전공'].str.contains("언론")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("커뮤니케이션")]['대학전공'].value_counts()

대학전공
커뮤니케이션    18
Name: count, dtype: int64

In [433]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("환경공")]
a.loc[:, '대학전공'] = '환1경공1학'
X_train_pre[X_train_pre['대학전공'].str.contains("환경공")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("환1경공1학")]['대학전공'].value_counts()

대학전공
환1경공1학    90
Name: count, dtype: int64

In [434]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("토목")]
a.loc[:, '대학전공'] = '토목'
X_train_pre[X_train_pre['대학전공'].str.contains("토목")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("토목")]['대학전공'].value_counts()

대학전공
토목    86
Name: count, dtype: int64

In [435]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("환경")]
a.loc[:, '대학전공'] = '환경'
X_train_pre[X_train_pre['대학전공'].str.contains("환경")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("환경")]['대학전공'].value_counts()

대학전공
환경    46
Name: count, dtype: int64

In [436]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("유통")]
a.loc[:, '대학전공'] = '유통'
X_train_pre[X_train_pre['대학전공'].str.contains("유통")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("유통")]['대학전공'].value_counts()

대학전공
유통    19
Name: count, dtype: int64

In [437]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("미생물")]
a.loc[:, '대학전공'] = '미생1물'
X_train_pre[X_train_pre['대학전공'].str.contains("미생물")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("미생1물")]['대학전공'].value_counts()

대학전공
미생1물    9
Name: count, dtype: int64

In [438]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("생물")]
a.loc[:, '대학전공'] = '생물'
X_train_pre[X_train_pre['대학전공'].str.contains("생물")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("생물")]['대학전공'].value_counts()

대학전공
생물    77
Name: count, dtype: int64

In [439]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("생명")]
a.loc[:, '대학전공'] = '바이오'
X_train_pre[X_train_pre['대학전공'].str.contains("생명")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("바이오")]['대학전공'].value_counts()

대학전공
바이오              56
바이오 메카트로닉스공학과     1
Name: count, dtype: int64

In [440]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("바이오")]
a.loc[:, '대학전공'] = '바이오'
X_train_pre[X_train_pre['대학전공'].str.contains("바이오")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("바이오")]['대학전공'].value_counts()

대학전공
바이오    57
Name: count, dtype: int64

In [441]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("조리")]
a.loc[:, '대학전공'] = '조리'
X_train_pre[X_train_pre['대학전공'].str.contains("조리")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("조리")]['대학전공'].value_counts()

대학전공
조리    7
Name: count, dtype: int64

In [442]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("호텔")]
a.loc[:, '대학전공'] = '호텔'
X_train_pre[X_train_pre['대학전공'].str.contains("호텔")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("호텔")]['대학전공'].value_counts()

대학전공
호텔    79
Name: count, dtype: int64

In [443]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("일어")]
a.loc[:, '대학전공'] = '일본'
X_train_pre[X_train_pre['대학전공'].str.contains("일어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("일본")]['대학전공'].value_counts()

대학전공
일본    206
Name: count, dtype: int64

In [444]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("가정")]
a.loc[:, '대학전공'] = '가정'
X_train_pre[X_train_pre['대학전공'].str.contains("가정")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("가정")]['대학전공'].value_counts()

대학전공
가정    44
Name: count, dtype: int64

In [445]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("가족")]
a.loc[:, '대학전공'] = '가정'
X_train_pre[X_train_pre['대학전공'].str.contains("가족")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("가정")]['대학전공'].value_counts()

대학전공
가정    45
Name: count, dtype: int64

In [446]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("신소재")]
a.loc[:, '대학전공'] = '신소재'
X_train_pre[X_train_pre['대학전공'].str.contains("신소재")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("신소재")]['대학전공'].value_counts()

대학전공
신소재    30
Name: count, dtype: int64

In [447]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("세라믹")]
a.loc[:, '대학전공'] = '세라믹'
X_train_pre[X_train_pre['대학전공'].str.contains("세라믹")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("세라믹")]['대학전공'].value_counts()

대학전공
세라믹    7
Name: count, dtype: int64

In [448]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("마케")]
a.loc[:, '대학전공'] = '마케팅'
X_train_pre[X_train_pre['대학전공'].str.contains("마케팅")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("마케팅")]['대학전공'].value_counts()

대학전공
마케팅    11
Name: count, dtype: int64

In [449]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("아동")]
a.loc[:, '대학전공'] = '아동'
X_train_pre[X_train_pre['대학전공'].str.contains("아동")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("아동")]['대학전공'].value_counts()

대학전공
아동    13
Name: count, dtype: int64

In [450]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("메카")]
a.loc[:, '대학전공'] = '메카트로닉스'
X_train_pre[X_train_pre['대학전공'].str.contains("메카")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("메카트로닉스")]['대학전공'].value_counts()

대학전공
메카트로닉스    9
Name: count, dtype: int64

In [451]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("전파")]
a.loc[:, '대학전공'] = '전파'
X_train_pre[X_train_pre['대학전공'].str.contains("전파")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("전파")]['대학전공'].value_counts()

대학전공
전파    11
Name: count, dtype: int64

In [452]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("제어")]
a.loc[:, '대학전공'] = '제어계측'
X_train_pre[X_train_pre['대학전공'].str.contains("제어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("제어계측")]['대학전공'].value_counts()

대학전공
제어계측    48
Name: count, dtype: int64

In [453]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("미디어")]
a.loc[:, '대학전공'] = '미디어'
X_train_pre[X_train_pre['대학전공'].str.contains("미디어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("미디어")]['대학전공'].value_counts()

대학전공
미디어    32
Name: count, dtype: int64

In [454]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("스페인")]
a.loc[:, '대학전공'] = '스페인'
X_train_pre[X_train_pre['대학전공'].str.contains("스페인")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("스페인")]['대학전공'].value_counts()

대학전공
스페인    2
Name: count, dtype: int64

In [455]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("언어")]
a.loc[:, '대학전공'] = '언어'
X_train_pre[X_train_pre['대학전공'].str.contains("언어")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("언어")]['대학전공'].value_counts()

대학전공
언어    1
Name: count, dtype: int64

In [456]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("소트")]
a.loc[:, '대학전공'] = '소프트웨어'
X_train_pre[X_train_pre['대학전공'].str.contains("소트")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("소프트웨어")]['대학전공'].value_counts()

대학전공
소프트웨어    6
Name: count, dtype: int64

In [457]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("디지털")]
a.loc[:, '대학전공'] = '디지털컨텐츠'
X_train_pre[X_train_pre['대학전공'].str.contains("디지털")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("디지털컨텐츠")]['대학전공'].value_counts()

대학전공
디지털컨텐츠    6
Name: count, dtype: int64

In [458]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("조형")]
a.loc[:, '대학전공'] = '조형'
X_train_pre[X_train_pre['대학전공'].str.contains("조형")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("조형")]['대학전공'].value_counts()

대학전공
조형    4
Name: count, dtype: int64

In [459]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("터정")]
a.loc[:, '대학전공'] = '컴퓨터정1보'
X_train_pre[X_train_pre['대학전공'].str.contains("터정")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("컴퓨터정1보")]['대학전공'].value_counts()

대학전공
컴퓨터정1보    29
Name: count, dtype: int64

In [460]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("터 정")]
a.loc[:, '대학전공'] = '컴퓨터정1보'
X_train_pre[X_train_pre['대학전공'].str.contains("터 정")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("컴퓨터정1보")]['대학전공'].value_counts()

대학전공
컴퓨터정1보    30
Name: count, dtype: int64

In [461]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("컴정")]
a.loc[:, '대학전공'] = '컴퓨터정1보'
X_train_pre[X_train_pre['대학전공'].str.contains("컴정")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("컴퓨터정1보")]['대학전공'].value_counts()

대학전공
컴퓨터정1보    30
Name: count, dtype: int64

In [462]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("컴퓨")]
a.loc[:, '대학전공'] = '컴퓨터'
X_train_pre[X_train_pre['대학전공'].str.contains("컴퓨")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("컴퓨터")]['대학전공'].value_counts()

대학전공
컴퓨터    682
Name: count, dtype: int64

In [463]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("약")]
a.loc[:, '대학전공'] = '제약'
X_train_pre[X_train_pre['대학전공'].str.contains("약")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("제약")]['대학전공'].value_counts()

대학전공
제약    8
Name: count, dtype: int64

In [464]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("관광경")]
a.loc[:, '대학전공'] = '관1광경1영'
X_train_pre[X_train_pre['대학전공'].str.contains("관광경")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("관1광경1영")]['대학전공'].value_counts()

대학전공
관1광경1영    103
Name: count, dtype: int64

In [465]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("관광 경")]
a.loc[:, '대학전공'] = '관1광경1영'
X_train_pre[X_train_pre['대학전공'].str.contains("관광 경")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("관1광경1영")]['대학전공'].value_counts()

대학전공
관1광경1영    103
Name: count, dtype: int64

In [466]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("통역")]
a.loc[:, '대학전공'] = '통역'
X_train_pre[X_train_pre['대학전공'].str.contains("통역")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("통역")]['대학전공'].value_counts()

대학전공
통역    8
Name: count, dtype: int64

In [467]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("관광")]
a.loc[:, '대학전공'] = '관광'
X_train_pre[X_train_pre['대학전공'].str.contains("관광")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("관광")]['대학전공'].value_counts()

대학전공
관광    34
Name: count, dtype: int64

In [468]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("네트")]
a.loc[:, '대학전공'] = '네트워크'
X_train_pre[X_train_pre['대학전공'].str.contains("네트")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("네트워크")]['대학전공'].value_counts()

대학전공
네트워크    1
Name: count, dtype: int64

In [469]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("원예")]
a.loc[:, '대학전공'] = '원예'
X_train_pre[X_train_pre['대학전공'].str.contains("원예")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("원예")]['대학전공'].value_counts()

대학전공
원예    8
Name: count, dtype: int64

In [470]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("지식")]
a.loc[:, '대학전공'] = '지식정1보'
X_train_pre[X_train_pre['대학전공'].str.contains("지식")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("지식정1보")]['대학전공'].value_counts()

대학전공
지식정1보    2
Name: count, dtype: int64

In [471]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("영상")]
a.loc[:, '대학전공'] = '영상'
X_train_pre[X_train_pre['대학전공'].str.contains("영상")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("영상")]['대학전공'].value_counts()

대학전공
영상    5
Name: count, dtype: int64

In [472]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("연극")]
a.loc[:, '대학전공'] = '연극영1화'
X_train_pre[X_train_pre['대학전공'].str.contains("연극")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("연극영1화")]['대학전공'].value_counts()

대학전공
연극영1화    4
Name: count, dtype: int64

In [473]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("영화")]
a.loc[:, '대학전공'] = '영상'
X_train_pre[X_train_pre['대학전공'].str.contains("영화")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("영상")]['대학전공'].value_counts()

대학전공
영상    5
Name: count, dtype: int64

In [474]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("미술")]
a.loc[:, '대학전공'] = '미술'
X_train_pre[X_train_pre['대학전공'].str.contains("미술")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("미술")]['대학전공'].value_counts()

대학전공
미술    40
Name: count, dtype: int64

In [475]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("보험")]
a.loc[:, '대학전공'] = '보험'
X_train_pre[X_train_pre['대학전공'].str.contains("보험")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("보험")]['대학전공'].value_counts()

대학전공
보험    32
Name: count, dtype: int64

In [476]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("항공")]
a.loc[:, '대학전공'] = '항공우주'
X_train_pre[X_train_pre['대학전공'].str.contains("항공")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("항공우주")]['대학전공'].value_counts()

대학전공
항공우주    4
Name: count, dtype: int64

In [477]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("윤리")]
a.loc[:, '대학전공'] = '철학'
X_train_pre[X_train_pre['대학전공'].str.contains("윤리")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("철학")]['대학전공'].value_counts()

대학전공
철학    70
Name: count, dtype: int64

In [478]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("인류")]
a.loc[:, '대학전공'] = '인류'
X_train_pre[X_train_pre['대학전공'].str.contains("인류")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("인류")]['대학전공'].value_counts()

대학전공
인류    3
Name: count, dtype: int64

In [479]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("수학")]
a.loc[:, '대학전공'] = '수학'
X_train_pre[X_train_pre['대학전공'].str.contains("수학")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("수학")]['대학전공'].value_counts()

대학전공
수학    98
Name: count, dtype: int64

In [480]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("전산")]
a.loc[:, '대학전공'] = '전산'
X_train_pre[X_train_pre['대학전공'].str.contains("전산")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("전산")]['대학전공'].value_counts()

대학전공
전산    97
Name: count, dtype: int64

In [481]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("선교")]
a.loc[:, '대학전공'] = '종교'
X_train_pre[X_train_pre['대학전공'].str.contains("선교")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("종교")]['대학전공'].value_counts()

대학전공
종교    2
Name: count, dtype: int64

In [482]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("기독")]
a.loc[:, '대학전공'] = '종교'
X_train_pre[X_train_pre['대학전공'].str.contains("기독")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("종교")]['대학전공'].value_counts()

대학전공
종교    4
Name: count, dtype: int64

In [483]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("해양")]
a.loc[:, '대학전공'] = '해양'
X_train_pre[X_train_pre['대학전공'].str.contains("해양")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("해양")]['대학전공'].value_counts()

대학전공
해양    22
Name: count, dtype: int64

In [484]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("부동산")]
a.loc[:, '대학전공'] = '부동산'
X_train_pre[X_train_pre['대학전공'].str.contains("부동산")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("부동산")]['대학전공'].value_counts()

대학전공
부동산    8
Name: count, dtype: int64

In [485]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("국제통상")]
a.loc[:, '대학전공'] = '국1제통상'
X_train_pre[X_train_pre['대학전공'].str.contains("국제통상")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("국1제통상")]['대학전공'].value_counts()

대학전공
국1제통상    345
Name: count, dtype: int64

In [486]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("보건")]
a.loc[:, '대학전공'] = '보건'
X_train_pre[X_train_pre['대학전공'].str.contains("보건")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("보건")]['대학전공'].value_counts()

대학전공
보건    15
Name: count, dtype: int64

In [487]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("과학")]
a.loc[:, '대학전공'] = '기타과학'
X_train_pre[X_train_pre['대학전공'].str.contains("과학")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("기타과학")]['대학전공'].value_counts()

대학전공
기타과학    19
Name: count, dtype: int64

In [488]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("신학")]
a.loc[:, '대학전공'] = '신학'
X_train_pre[X_train_pre['대학전공'].str.contains("신학")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("신학")]['대학전공'].value_counts()

대학전공
신학    21
Name: count, dtype: int64

In [489]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("종교")]
a.loc[:, '대학전공'] = '신학'
X_train_pre[X_train_pre['대학전공'].str.contains("종교")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("신학")]['대학전공'].value_counts()

대학전공
신학    25
Name: count, dtype: int64

In [490]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("농")]
a.loc[:, '대학전공'] = '농업'
X_train_pre[X_train_pre['대학전공'].str.contains("농")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("농업")]['대학전공'].value_counts()

대학전공
농업    11
Name: count, dtype: int64

In [491]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("소재")]
a.loc[:, '대학전공'] = '신소재'
X_train_pre[X_train_pre['대학전공'].str.contains("소재")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("신소재")]['대학전공'].value_counts()

대학전공
신소재    36
Name: count, dtype: int64

In [492]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("정치")]
a.loc[:, '대학전공'] = '정치외교'
X_train_pre[X_train_pre['대학전공'].str.contains("정치")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("정치외교")]['대학전공'].value_counts()

대학전공
정치외교    89
Name: count, dtype: int64

In [493]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("정외")]
a.loc[:, '대학전공'] = '정치외교'
X_train_pre[X_train_pre['대학전공'].str.contains("정외")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("정치외교")]['대학전공'].value_counts()

대학전공
정치외교    90
Name: count, dtype: int64

In [494]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("산업정")]
a.loc[:, '대학전공'] = '산1업정1보'
X_train_pre[X_train_pre['대학전공'].str.contains("산업정")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("산1업정1보")]['대학전공'].value_counts()

대학전공
산1업정1보    58
Name: count, dtype: int64

In [495]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("도시")]
a.loc[:, '대학전공'] = '도시'
X_train_pre[X_train_pre['대학전공'].str.contains("도시")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("도시")]['대학전공'].value_counts()

대학전공
도시    34
Name: count, dtype: int64

In [496]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("산업공")]
a.loc[:, '대학전공'] = '산업공1학'
X_train_pre[X_train_pre['대학전공'].str.contains("산업공")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("산업공1학")]['대학전공'].value_counts()

대학전공
산업공1학    245
Name: count, dtype: int64

In [497]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("금속")]
a.loc[:, '대학전공'] = '금속'
X_train_pre[X_train_pre['대학전공'].str.contains("금속")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("금속")]['대학전공'].value_counts()

대학전공
금속    68
Name: count, dtype: int64

In [498]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("유전")]
a.loc[:, '대학전공'] = '바이오'
X_train_pre[X_train_pre['대학전공'].str.contains("유전")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("바이오")]['대학전공'].value_counts()

대학전공
바이오    74
Name: count, dtype: int64

In [499]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("산업경")]
a.loc[:, '대학전공'] = '산1업경1영'
X_train_pre[X_train_pre['대학전공'].str.contains("산업경")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("산1업경1영")]['대학전공'].value_counts()

대학전공
산1업경1영    20
Name: count, dtype: int64

In [500]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("건설")]
a.loc[:, '대학전공'] = '건축'
X_train_pre[X_train_pre['대학전공'].str.contains("건설")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("건축")]['대학전공'].value_counts()

대학전공
건축    136
Name: count, dtype: int64

In [501]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("충")]
a.loc[:, '대학전공'] = '건축'
X_train_pre[X_train_pre['대학전공'].str.contains("충")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("건축")]['대학전공'].value_counts()

대학전공
건축    137
Name: count, dtype: int64

In [502]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("건강")]
a.loc[:, '대학전공'] = '보건'
X_train_pre[X_train_pre['대학전공'].str.contains("건강")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("보건")]['대학전공'].value_counts()

대학전공
보건    19
Name: count, dtype: int64

In [503]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("화학")]
a.loc[:, '대학전공'] = '화학'
X_train_pre[X_train_pre['대학전공'].str.contains("화학")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("화학")]['대학전공'].value_counts()

대학전공
화학    172
Name: count, dtype: int64

In [504]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("물리")]
a.loc[:, '대학전공'] = '물리'
X_train_pre[X_train_pre['대학전공'].str.contains("물리")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("물리")]['대학전공'].value_counts()

대학전공
물리    103
Name: count, dtype: int64

In [505]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("유도")]
a.loc[:, '대학전공'] = '유도'
X_train_pre[X_train_pre['대학전공'].str.contains("유도")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("유도")]['대학전공'].value_counts()

대학전공
유도    10
Name: count, dtype: int64

In [506]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("체욱")]
a.loc[:, '대학전공'] = '체육'
X_train_pre[X_train_pre['대학전공'].str.contains("체욱")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("체육")]['대학전공'].value_counts()

대학전공
체육    98
Name: count, dtype: int64

In [507]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("섬유공")]
a.loc[:, '대학전공'] = '섬1유공1학'
X_train_pre[X_train_pre['대학전공'].str.contains("섬유공")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("섬1유공1학")]['대학전공'].value_counts()

대학전공
섬1유공1학    24
Name: count, dtype: int64

In [508]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("섬유")]
a.loc[:, '대학전공'] = '섬유예술'
X_train_pre[X_train_pre['대학전공'].str.contains("섬유")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("섬유예술")]['대학전공'].value_counts()

대학전공
섬유예술    26
Name: count, dtype: int64

In [509]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("정보공학")]
a.loc[:, '대학전공'] = '정보공1학'
X_train_pre[X_train_pre['대학전공'].str.contains("정보공학")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("정보공1학")]['대학전공'].value_counts()

대학전공
정보공1학    29
Name: count, dtype: int64

In [510]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("분자")]
a.loc[:, '대학전공'] = '고분자'
X_train_pre[X_train_pre['대학전공'].str.contains("분자")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("고분자")]['대학전공'].value_counts()

대학전공
고분자    16
Name: count, dtype: int64

In [511]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("공학")]
a.loc[:, '대학전공'] = '기타공학'
X_train_pre[X_train_pre['대학전공'].str.contains("공학")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("기타공학")]['대학전공'].value_counts()

대학전공
기타공학    56
Name: count, dtype: int64

In [512]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("금융")]
a.loc[:, '대학전공'] = '금융'
X_train_pre[X_train_pre['대학전공'].str.contains("금융")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("금융")]['대학전공'].value_counts()

대학전공
금융    11
Name: count, dtype: int64

In [513]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("비서")]
a.loc[:, '대학전공'] = '비서'
X_train_pre[X_train_pre['대학전공'].str.contains("비서")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("비서")]['대학전공'].value_counts()

대학전공
비서    8
Name: count, dtype: int64

In [514]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("축산")]
a.loc[:, '대학전공'] = '축산'
X_train_pre[X_train_pre['대학전공'].str.contains("축산")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("축산")]['대학전공'].value_counts()

대학전공
축산    6
Name: count, dtype: int64

In [515]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("지리")]
a.loc[:, '대학전공'] = '지리'
X_train_pre[X_train_pre['대학전공'].str.contains("지리")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("지리")]['대학전공'].value_counts()

대학전공
지리    10
Name: count, dtype: int64

In [516]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("무용")]
a.loc[:, '대학전공'] = '무용'
X_train_pre[X_train_pre['대학전공'].str.contains("무용")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("무용")]['대학전공'].value_counts()

대학전공
무용    19
Name: count, dtype: int64

In [517]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("양화")]
a.loc[:, '대학전공'] = '미술'
X_train_pre[X_train_pre['대학전공'].str.contains("양화")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("미술")]['대학전공'].value_counts()

대학전공
미술    78
Name: count, dtype: int64

In [518]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("정보경영")]
a.loc[:, '대학전공'] = '경영정보'
X_train_pre[X_train_pre['대학전공'].str.contains("정보경영")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("경영정보")]['대학전공'].value_counts()

대학전공
경영정보    78
Name: count, dtype: int64

In [519]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("경영정보")]
a.loc[:, '대학전공'] = '경1영정1보'
X_train_pre[X_train_pre['대학전공'].str.contains("경영정보")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("경1영정1보")]['대학전공'].value_counts()

대학전공
경1영정1보    78
Name: count, dtype: int64

In [520]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("경영")]
a.loc[:, '대학전공'] = '경영'
X_train_pre[X_train_pre['대학전공'].str.contains("경영")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("경영")]['대학전공'].value_counts()

대학전공
경영    1022
Name: count, dtype: int64

In [521]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("산업재산")]
a.loc[:, '대학전공'] = '산업재산권'
X_train_pre[X_train_pre['대학전공'].str.contains("산업재산")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("산업재산권")]['대학전공'].value_counts()

대학전공
산업재산권    1
Name: count, dtype: int64

In [522]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("정보공")]
a.loc[:, '대학전공'] = '정보공1학'
X_train_pre[X_train_pre['대학전공'].str.contains("정보공")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("정보공1학")]['대학전공'].value_counts()

대학전공
정보공1학    29
Name: count, dtype: int64

In [523]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("정보공1학")]
a.loc[:, '대학전공'] = '정1보공1학'
X_train_pre[X_train_pre['대학전공'].str.contains("정보공1학")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("정1보공1학")]['대학전공'].value_counts()

대학전공
정1보공1학    29
Name: count, dtype: int64

In [524]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("정보")]
a.loc[:, '대학전공'] = '기타정보'
X_train_pre[X_train_pre['대학전공'].str.contains("정보")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("기타정보")]['대학전공'].value_counts()

대학전공
기타정보    28
Name: count, dtype: int64

In [525]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("도예")]
a.loc[:, '대학전공'] = '도예'
X_train_pre[X_train_pre['대학전공'].str.contains("도예")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("도예")]['대학전공'].value_counts()

대학전공
도예    5
Name: count, dtype: int64

In [526]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("화공")]
a.loc[:, '대학전공'] = '화1학공'
X_train_pre[X_train_pre['대학전공'].str.contains("화공")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("화1학공")]['대학전공'].value_counts()

대학전공
화1학공1학    156
화1학공        3
Name: count, dtype: int64

In [527]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("화1학공")]
a.loc[:, '대학전공'] = '화1학공1학'
X_train_pre[X_train_pre['대학전공'].str.contains("화1학공")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("화1학공1학")]['대학전공'].value_counts()

대학전공
화1학공1학    159
Name: count, dtype: int64

In [528]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("국제")]
a.loc[:, '대학전공'] = '국제'
X_train_pre[X_train_pre['대학전공'].str.contains("국제")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("국제")]['대학전공'].value_counts()

대학전공
국제    26
Name: count, dtype: int64

In [529]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("문예창작")]
a.loc[:, '대학전공'] = '문예창작'
X_train_pre[X_train_pre['대학전공'].str.contains("문예창작")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("문예창작")]['대학전공'].value_counts()

대학전공
문예창작    16
Name: count, dtype: int64

In [530]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("의과")]
a.loc[:, '대학전공'] = '의예'
X_train_pre[X_train_pre['대학전공'].str.contains("의과")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("의예")]['대학전공'].value_counts()

Series([], Name: count, dtype: int64)

In [531]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("의예")]
a.loc[:, '대학전공'] = '의예'
X_train_pre[X_train_pre['대학전공'].str.contains("의예")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("의예")]['대학전공'].value_counts()

Series([], Name: count, dtype: int64)

In [532]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("병")]
a.loc[:, '대학전공'] = '의예'
X_train_pre[X_train_pre['대학전공'].str.contains("병")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("의예")]['대학전공'].value_counts()

대학전공
의예    1
Name: count, dtype: int64

In [533]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("도예")]
a.loc[:, '대학전공'] = '도예'
X_train_pre[X_train_pre['대학전공'].str.contains("도예")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("도예")]['대학전공'].value_counts()

대학전공
도예    5
Name: count, dtype: int64

In [534]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("한국화")]
a.loc[:, '대학전공'] = '미술'
X_train_pre[X_train_pre['대학전공'].str.contains("한국화")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("미술")]['대학전공'].value_counts()

대학전공
미술    81
Name: count, dtype: int64

In [535]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("문화")]
a.loc[:, '대학전공'] = '문화재'
X_train_pre[X_train_pre['대학전공'].str.contains("문화")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("문화재")]['대학전공'].value_counts()

대학전공
문화재    7
Name: count, dtype: int64

In [536]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("조경")]
a.loc[:, '대학전공'] = '조경'
X_train_pre[X_train_pre['대학전공'].str.contains("조경")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("조경")]['대학전공'].value_counts()

대학전공
조경    13
Name: count, dtype: int64

In [537]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("간호")]
a.loc[:, '대학전공'] = '간호'
X_train_pre[X_train_pre['대학전공'].str.contains("간호")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("간호")]['대학전공'].value_counts()

대학전공
간호    12
Name: count, dtype: int64

In [538]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("지역")]
a.loc[:, '대학전공'] = '지역개발'
X_train_pre[X_train_pre['대학전공'].str.contains("지역")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("지역개발")]['대학전공'].value_counts()

대학전공
지역개발    18
Name: count, dtype: int64

In [539]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("지질")]
a.loc[:, '대학전공'] = '지질'
X_train_pre[X_train_pre['대학전공'].str.contains("지질")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("지질")]['대학전공'].value_counts()

대학전공
지질    4
Name: count, dtype: int64

In [540]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("영양")]
a.loc[:, '대학전공'] = '식1품영양'
X_train_pre[X_train_pre['대학전공'].str.contains("영양")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("식1품영양")]['대학전공'].value_counts()

대학전공
식1품영양    69
Name: count, dtype: int64

In [541]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("성악")]
a.loc[:, '대학전공'] = '성아ㄱ'
X_train_pre[X_train_pre['대학전공'].str.contains("성악")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("성아ㄱ")]['대학전공'].value_counts()

대학전공
성아ㄱ    8
Name: count, dtype: int64

In [542]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("악")]
a.loc[:, '대학전공'] = '음악'
X_train_pre[X_train_pre['대학전공'].str.contains("악")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("음악")]['대학전공'].value_counts()

대학전공
음악    13
Name: count, dtype: int64

In [543]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("조소")]
a.loc[:, '대학전공'] = '조소'
X_train_pre[X_train_pre['대학전공'].str.contains("조소")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("조소")]['대학전공'].value_counts()

대학전공
조소    13
Name: count, dtype: int64

In [544]:
a = X_train_pre[X_train_pre['대학전공'].str.contains("회화")]
a.loc[:, '대학전공'] = '회화'
X_train_pre[X_train_pre['대학전공'].str.contains("회화")] = a
X_train_pre[X_train_pre['대학전공'].str.contains("회화")]['대학전공'].value_counts()

대학전공
회화    8
Name: count, dtype: int64

In [545]:
a = X_train_pre['대학전공'].value_counts().loc[lambda x : x <= 3]
기타 = list(a.index)

In [546]:
X_train_pre.대학전공 = X_train_pre.대학전공.replace(기타,'기타')

In [547]:
X_train_pre['대학전공'] = X_train_pre['대학전공'].str.replace('1','')

In [548]:
X_train_pre['대학전공'] = X_train_pre['대학전공'].str.replace('아ㄱ','악')

In [549]:
X_test.대학전공 = X_train_pre.대학전공

## 자격증

In [550]:
X_train.자격증 = X_train.자격증.replace("有","1")
X_train.자격증 = X_train.자격증.replace("無","0")

X_test.자격증 = X_test.자격증.replace("有","1")
X_test.자격증 = X_test.자격증.replace("無","0")

In [551]:
X_train.대학전공

1679         토목
5182       전자계산
3116        경영학
8390        회계학
4496        경영학
          ...  
1097     컴퓨터공학과
15843       회계학
5385      제품디자인
11019      전자공학
3137       관광학과
Name: 대학전공, Length: 16570, dtype: object

# 추출

In [552]:
X_train.reset_index().drop('index',axis=1)

,ID,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,대학성적
0,SID167237,건설·기계·전기·전자,건설·토목·건축설비,기타,0,NaN,서울,연세대학교,토목,TOEIC,1,70.0
1,SID316308,IT·게임,시스템분석·설계·컨설턴트·ERP,기타,0,NaN,대전,한밭대학교,전자계산,없음,1,80.0
2,SID220787,경영·기획·회계·사무,경리·회계,ac,1275,NaN,서울,이화여자대학교,경영학,TOEIC,1,70.0
3,SID470405,경영·기획·회계·사무,경리·회계,ac,1670,NaN,강원,상지대학교,회계학,없음,1,80.0
4,SID278740,경영·기획·회계·사무,경리·회계,ac,0,NaN,서울,이화여자대학교,경영학,없음,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16565,SID145408,건설·기계·전기·전자,전기·전자·제어,휴대,485,NaN,서울,동아대학교,컴퓨터공학과,TOEIC,1,60.0
16566,SID819286,경영·기획·회계·사무,구매·자재사무,휴대,60,정규직,인천,세명대학교,회계학,없음,0,70.0
16567,SID326624,디자인,제품·산업디자인,휴대,0,NaN,서울,한밭대학교,제품디자인,없음,1,80.0
16568,SID579826,경영·기획·회계·사무,마케팅·마케팅기획,휴대,4045,정규직,서울,제주대학교,전자공학,TOEIC,1,70.0


In [553]:
X_test.reset_index().drop('index',axis=1)

,ID,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,대학성적
0,SID16772,IT·게임,시스템분석·설계·컨설턴트·ERP,ab,0,NaN,서울,성신여자대학교,전산,없음,0,80.0
1,SID171384,IT·게임,시스템분석·설계·컨설턴트·ERP,ab,0,NaN,서울,서울여자대학교,컴퓨터,기타,1,70.0
2,SID181009,경영·기획·회계·사무,상담·고객지원,ac,730,정규직,서울,성신여자대학교,공예,없음,1,70.0
3,SID625415,경영·기획·회계·사무,경리·회계,ac,2555,정규직,서울,중앙대학교,회계,없음,0,80.0
4,SID230359,경영·기획·회계·사무,경리·회계,ac,0,정규직,서울,세종대학교,영어,TOEIC,1,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11043,SID240843,경영·기획·회계·사무,무역·수출입·운송사무,기타,0,정규직,서울,중앙대학교,교육,없음,1,70.0
11044,SID22550,경영·기획·회계·사무,무역·수출입·운송사무,기타,0,NaN,대전,군산대학교,국제통상,없음,1,90.0
11045,SID360804,경영·기획·회계·사무,구매·자재사무,휴대,0,NaN,경기,경기대학교,신소재,TOEIC,0,60.0
11046,SID638241,IT·게임,"응용프로그래머(Unix,Linux,Java)",휴대,1185,정규직,서울,성균관대학교,통신,없음,0,80.0


In [ ]:
X_train.to_csv("data/X_train_pre_2.csv",index=False)

In [ ]:
X_test.to_csv("data/X_test_pre_2.csv",index=False)

# 모델링

In [554]:
X_train = pd.read_csv("/content/drive/MyDrive/머신러닝 수업/data/X_train_pre_2.csv")
X_test = pd.read_csv("/content/drive/MyDrive/머신러닝 수업/data/X_test_pre_2.csv")
y_train = pd.read_csv("/content/drive/MyDrive/머신러닝 수업/data/y_train.csv")

In [555]:
X_train = X_train.sort_values(by='ID').reset_index().drop(columns='index')

In [556]:
y_train

,ID,Salary
0,SID100109,2100
1,SID100255,2100
2,SID100284,1900
3,SID100315,2500
4,SID100374,1900
...,...,...
16565,SID99558,1700
16566,SID99611,2100
16567,SID99741,1700
16568,SID99756,1700


In [557]:
CATBOOST_VERSION = 1.0

In [558]:
y_train = y_train.Salary

In [559]:
numeric_features = ['대학성적','근무경력']
categorical_features = ['직종','세부직종','직무태그','근무형태','근무지역','출신대학','대학전공','어학시험','자격증']

X_train = X_train[numeric_features+categorical_features]  # 순서 주의!!!
X_test = X_test[numeric_features+categorical_features]

## 변환 파이프라인

In [560]:
def remove_outlier(X, q=0.05):
    df = pd.DataFrame(X)
    return df.apply(lambda x: x.clip(x.quantile(q), x.quantile(1-q)), axis=0).values

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("outlier", FunctionTransformer(remove_outlier, kw_args={'q':0.05})), # 함수를 전처리기로 변환하여 sklearn에 없는 새로운 전처리기를 만듬
        ("scaler", PowerTransformer()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=99999, dtype=object)),
    ]
)


column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

preprocessor = Pipeline(
    steps=[
        ("column", column_transformer),
        ("selector", SelectPercentile(percentile=100)), # SelectPercentile(percentile=100)), 피쳐고르는비율지정
#        ("dataframe", FunctionTransformer(lambda x: pd.DataFrame(x, columns=numeric_features+categorical_features))),
    ]
)

set_config(display="diagram")  # To view the text pipeline, change to display='text'.
preprocessor

Pipeline(steps=[('column',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('outlier',
                                                                   FunctionTransformer(func=<function remove_outlier at 0x7ff783136680>,
                                                                                       kw_args={'q': 0.05})),
                                                                  ('scaler',
                                                                   PowerTransformer())]),
                                                  ['대학성적', '근무경력']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder(dtype=<class 'object'>,
                                                                                  handle_unknown='use_encoded_value',
                                                                                  unknown_value=99999))]),
                                                  ['직종', '세부직종', '직무태그', '근무형태',
                                                   '근무지역', '출신대학', '대학전공',
                                                   '어학시험', '자격증'])])),
                ('selector', SelectPercentile(percentile=100))])

In [561]:
preprocessor.fit(X_train, y_train)
mask = preprocessor.steps[-1][-1].get_support(); new_features = X_train.columns[mask]

In [562]:
X_train = pd.DataFrame(preprocessor.transform(X_train), columns=new_features)
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=new_features)

In [563]:
numeric_features = list(set(numeric_features) & set(new_features))
categorical_features = list(set(categorical_features) & set(new_features))

## GridSearch - 최적 파라미터 셀렉

In [564]:
from sklearn.model_selection import GridSearchCV

model = CatBoostRegressor()
parameters = {'depth' : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [30, 50, 100]
              }

grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)

0:	learn: 1190.9877496	total: 4.41ms	remaining: 437ms
1:	learn: 1154.4661244	total: 9.38ms	remaining: 460ms
2:	learn: 1123.8236258	total: 12.9ms	remaining: 416ms
3:	learn: 1096.3566739	total: 16.4ms	remaining: 394ms
4:	learn: 1073.3285777	total: 20.1ms	remaining: 383ms
5:	learn: 1055.0931756	total: 23.5ms	remaining: 369ms
6:	learn: 1038.2971045	total: 27.2ms	remaining: 361ms
7:	learn: 1025.1476170	total: 30.8ms	remaining: 354ms
8:	learn: 1012.1542744	total: 34.3ms	remaining: 347ms
9:	learn: 1002.1156515	total: 37.9ms	remaining: 341ms
10:	learn: 994.1781702	total: 41.6ms	remaining: 336ms
11:	learn: 986.6245113	total: 45.1ms	remaining: 331ms
12:	learn: 980.3903919	total: 49ms	remaining: 328ms
13:	learn: 975.0619182	total: 52.7ms	remaining: 323ms
14:	learn: 967.6462931	total: 56.4ms	remaining: 320ms
15:	learn: 962.2043953	total: 60.3ms	remaining: 316ms
16:	learn: 958.9717110	total: 63.9ms	remaining: 312ms
17:	learn: 955.1700574	total: 67.4ms	remaining: 307ms
18:	learn: 948.0088851	total: 

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostRegressor object at 0x7ff7802882b0>,
             n_jobs=-1,
             param_grid={'depth': [6, 8, 10], 'iterations': [30, 50, 100],
                         'learning_rate': [0.01, 0.05, 0.1]})

In [565]:
grid.best_params_

{'depth': 6, 'iterations': 100, 'learning_rate': 0.1}

## catboost 모델

In [577]:
scores = []  # CV 결과 저장
oof_pred = np.zeros(X_test.shape[0])  # OOF 저장
kfold = KFold(n_splits=5, shuffle=True, random_state=0) # K-Folds cross-validator #n_splits=5 > k 5개 쪼개기, #shuffle > 데이터 섞기 여부. TRUE가 좋음,

for train_index, valid_index in kfold.split(X_train, y_train): #kfold.split : 학습데이터가 5개로 쪼개진다. valid_index >test, train_index>train
    # 학습/검증 데이터 분할
    train_x, valid_x = X_train.iloc[train_index], X_train.iloc[valid_index]
    train_y, valid_y = y_train.iloc[train_index], y_train.iloc[valid_index]

    # 범주형피처 직접 처리와 Early stopping을 사용하여 CatBoost 모델링
    model = CatBoostRegressor(cat_features=categorical_features, verbose=False, random_state=0, **grid.best_params_) #cat_features=categorical_features : 범주형 피쳐 지정해주면 지가 알아서 해줌
    model.fit(train_x, train_y, #4개의 fold로 학습,
              eval_set=[(valid_x,valid_y)],
              early_stopping_rounds=100, #early_stopping_rounds=100 과대적합 줄이기 위함
             )

    # CV 스코어 계산 및 저장
    rmse = np.sqrt(mean_squared_error(valid_y, model.predict(valid_x)))
    scores.append(rmse)

    # OOF 예측값 저장
    oof_pred += model.predict(X_test) / kfold.get_n_splits() #oof와 corssvalidation 한꺼번에 가능.

In [579]:
scores = np.array(scores)
print("CV scores: ", scores)
print("CV mean = %.2f" % scores.mean(), "with std = %.2f" % scores.std())

CV scores:  [865.43451797 846.03093425 875.26949722 835.01248508 848.86497392]
CV mean = 854.12 with std = 14.38
